# [1]토스앱 리뷰 수집

In [1]:
!pip install sqlalchemy pymysql

In [6]:
!pip install sqlalchemy pymysql python-dotenv

In [12]:
# 라이브러리
import sys
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from datetime import datetime, timedelta
import pandas as pd
import time

target_dir = os.path.abspath('../../') # 05data_scraping 폴더 위치
if target_dir not in sys.path:
    sys.path.append(target_dir)

# dbio > to_db 추가
from dbio import to_db
# module_path = os.path.abspath('../../dbio.py')

# Chrome 옵션 설정
options = Options()
options.add_argument("--window-size=1280,900")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# ChromeDriver 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.set_page_load_timeout(30)

try:
    driver.get("https://play.google.com/store/apps/details?id=viva.republica.toss")
    print(driver.title)
finally:
    pass

wait = WebDriverWait(driver, 10)

# 자바스트립트로 윈도우를 0-1200px까지 스크롤 내리기
driver.execute_script("window.scrollTo(0, 1200)")

# 평점 및 리뷰 옆의 -> 버튼 찾아서 클릭하기
btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="평점 및 리뷰 자세히 알아보기"]')))
btn.click()

# 최신순으로 리뷰를 정렬하기 위해 버튼 클릭
button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#sortBy_1")))
button.click()
time.sleep(5)

# 최신을 찾아 클릭
button2 = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'span[aria-label="최신"]')))
button2.click()
time.sleep(5)

#############################
# 팝업된 리뷰창에서 데이터 수집하기 #
#############################

# 오늘 날짜 추출 후, 1달 전 날짜 계산하여 수집할 날짜 범위 정하기
today = datetime.today()
end_date = today - timedelta(days=28)

while True:
    
    # 리뷰창 스크롤 내려서 과거 리뷰 로딩
    driver.execute_script("document.querySelector('.fysCi.Vk3ZVd').scrollBy(0, 1000)")
    time.sleep(1)
    
    # 리뷰 가장 마지막의 날짜 추출
    last_date = driver.find_elements(By.CSS_SELECTOR, ".bp9Aid")[-1].text
    last_date = last_date.replace("년 ", "-").replace("월 ", "-").replace("일", "")
    last_date = datetime.strptime(last_date, "%Y-%m-%d")
    n_reviews = len(driver.find_elements(By.CSS_SELECTOR, ".bp9Aid"))
    print("리뷰개수:", n_reviews, "end_date", end_date, "last_date", last_date, end="\r")
    if last_date < end_date:
        break

all_reviews = []

# 스크롤 멈춘 후 데이터 수집
for item in driver.find_elements(By.CSS_SELECTOR, ".RHo1pe"):
    result = {}
    
    # 리뷰에서 날짜 추출
    date = item.find_element(By.CSS_SELECTOR, ".bp9Aid").text
    date = date.replace("년 ", "-").replace("월 ", "-").replace("일", "")
    review_date = datetime.strptime(date, "%Y-%m-%d")
    
    # 평점 추출
    rating = item.find_element(By.CSS_SELECTOR, ".iXRFPc").get_attribute("aria-label")
    rating = rating.split()[3][0]
    
    # 리뷰 글 추출
    review_text = item.find_element(By.CSS_SELECTOR, ".h3YV2d").text
    
    result["date"] = review_date
    result['rating'] = rating
    result['review_text'] = review_text
    all_reviews.append(result)
    
df = pd.DataFrame(all_reviews)
display(df) 
to_db("bank_app_reviews", "toss_reviews", df)

토스 - Google Play 앱
리뷰개수: 463 end_date 2026-01-05 02:41:30.710025 last_date 2026-01-05 00:00:00

,date,rating,review_text
0,2026-01-31,2,내소비 월평균 막대그래프에서 과거 소비액 표시가 잘못 나타나고 있는 오류가 있습니다...
1,2026-01-31,4,토스는 아주~ 편하게 잘 쓰고있답니다 근데ㅋㅋ 쇼핑몰 첨 가봤는데 위로(↑)가는게 ...
2,2026-01-31,2,포인트 출금하는데 수수료를 10%?? 심하네요.
3,2026-01-31,5,많은도움이되고제미있기도하고머리휘전도되고싸이는제미도있고좀더다항한이이탬좀내씀하내요외르들...
4,2026-01-31,2,아니 어플 관리 안하시나 어느날부터 앱이 버벅대는데 개선이 안되네
...,...,...,...
455,2026-01-05,1,설치 중 취소 누르니 무한루핑 걸리며 설치도 제거도 되지 않아요
456,2026-01-05,1,앱 내 광고를 끄면 또 떠서 이중 광고하네요
457,2026-01-05,1,토스 은행앱으로 사용하는분중에 많은 분들이 월별 자산 변화량 그래프로 보여 주는거 ...
458,2026-01-05,2,업데이트가 99%에서 계속 멈춤. 이번 버전은 문제가 있는 것 같음.


ValueError: invalid literal for int() with base 10: 'None'

In [13]:
to_db("bank_app_reviews", "toss_reviews", df)

ValueError: invalid literal for int() with base 10: 'None'

In [15]:
import sys
import os
import time
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

# dbio 경로 설정 및 임포트
target_dir = os.path.abspath('../../')
if target_dir not in sys.path:
    sys.path.append(target_dir)
from dbio import to_db

# 브라우저 설정
options = Options()
options.add_argument("--window-size=1280,1000")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 10)

try:
    # (1)~(4) 구글플레이 검색 및 "금융" 입력 후 엔터
    driver.get("https://play.google.com/store/apps")
    search_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="검색"]')))
    search_button.click()
    
    search_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[aria-label="앱 검색"]')))
    search_input.send_keys("금융")
    search_input.send_keys(Keys.ENTER)
    time.sleep(3)

    # (5) 검색 결과 상위 5개 금융앱 URL 및 이름 추출
    apps = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.Si6A0c')))[:5]
    app_list = []
    for app in apps:
        app_url = app.get_attribute('href')
        app_name = app.find_element(By.CSS_SELECTOR, '.DdYf3c').text
        # 테이블명 사용을 위해 특수문자 제거 및 영문 변환(수동 지정 권장하나 여기선 슬라이싱 처리)
        table_name = "".join(filter(str.isalnum, app_url.split('id=')[-1].split('.')[-1])) + "_reviews"
        app_list.append({'name': app_name, 'url': app_url, 'table': table_name})

    # 수집 기간 설정: 1월 1일 ~ 1월 30일
    start_target = datetime(2026, 1, 30)
    end_target = datetime(2026, 1, 1)

    # (5)-(6) 각 앱별 반복 수집 및 저장
    for app_info in app_list:
        print(f"\n[{app_info['name']}] 수집 시작...")
        driver.get(app_info['url'])
        time.sleep(2)
        
        # 리뷰 페이지 진입
        driver.execute_script("window.scrollTo(0, 1000)")
        btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="평점 및 리뷰 자세히 알아보기"]')))
        btn.click()
        
        # 최신순 정렬
        sort_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#sortBy_1")))
        sort_btn.click()
        time.sleep(1)
        newest = wait.until(EC.element_to_be_clickable((By.XPATH, '//span[text()="최신순"]')))
        newest.click()
        time.sleep(3)

        # 팝업 내 스크롤 및 기간 필터링
        popup = driver.find_element(By.CSS_SELECTOR, '.fysCi.Vk3ZVd')
        all_reviews = []
        
        while True:
            driver.execute_script("arguments[0].scrollBy(0, 2000)", popup)
            time.sleep(1.5)
            
            dates = driver.find_elements(By.CSS_SELECTOR, ".bp9Aid")
            if not dates: continue
            
            # 마지막 날짜 확인
            last_date_str = dates[-1].text.replace("년 ", "-").replace("월 ", "-").replace("일", "").replace(". ","-").replace(".","")
            last_date_obj = datetime.strptime(last_date_raw, "%Y-%m-%d") if '-' in last_date_raw else datetime.now()

            if last_date_obj < end_target:
                break
        
        # 데이터 파싱
        review_elements = driver.find_elements(By.CSS_SELECTOR, ".RHo1pe")
        for item in review_elements:
            d_str = item.find_element(By.CSS_SELECTOR, ".bp9Aid").text.replace("년 ", "-").replace("월 ", "-").replace("일", "").replace(". ","-").replace(".","")
            d_obj = datetime.strptime(d_str, "%Y-%m-%d")
            
            # 1월 데이터만 선별
            if end_target <= d_obj <= start_target:
                all_reviews.append({
                    "date": d_obj,
                    "rating": item.find_element(By.CSS_SELECTOR, ".iXRFPc").get_attribute("aria-label").split()[[3]][[0]],
                    "review_text": item.find_element(By.CSS_SELECTOR, ".h3YV2d").text
                })
        
        # DB 저장
        if all_reviews:
            df = pd.DataFrame(all_reviews)
            to_db("bank_app_reviews", app_info['table'], df)
            print(f"-> {len(df)}건 저장 완료 (테이블: {app_info['table']})")

finally:
    driver.quit()

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001033bafd4 cxxbridge1$str$ptr + 3095476
1   chromedriver                        0x00000001033b33ec cxxbridge1$str$ptr + 3063756
2   chromedriver                        0x0000000102e9683c _RNvCs5DBLTqoOdVp_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 75008
3   chromedriver                        0x0000000102edfd68 _RNvCs5DBLTqoOdVp_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 375340
4   chromedriver                        0x0000000102f1efdc _RNvCs5DBLTqoOdVp_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 634016
5   chromedriver                        0x0000000102ed406c _RNvCs5DBLTqoOdVp_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 326960
6   chromedriver                        0x00000001033795dc cxxbridge1$str$ptr + 2826684
7   chromedriver                        0x000000010337cd1c cxxbridge1$str$ptr + 2840828
8   chromedriver                        0x000000010335ebec cxxbridge1$str